# Homework - Week 4
```
Put the "Homework 4" folder inside the main "Python_DA" training folder, the path of this folder will be the base path now.
Do the following transformation/cleaning tasks on ALL the source files within the "Input" folder ("Orders_2011.csv", "Orders_2012.csv", "Orders_2013.csv", "Orders_2014.csv"):

Tasks:
1. Use your "Read_parameters.ipynb" external program to read the parameter tables from "Parameters_Homework 4.xlsx"
2. Loop through the input files of the input folder, use your "Read_csv.ipynb" external program to read all of them, and append them into 1 dataframe
3. Create a "Clean_order_data.ipynb" external program, from only the data cleaning part of the Homework2 program (creating dimension tables is not needed now)
4. Run this "Clean_order_data.ipynb" external program to clean the dataframe
5. As the "Size" data in the 1st parameter table can be numeric and text values mixed, convert "Size" column to STRING data type (otherwise only the texts will be matched, e.g. "XL")
6. Filter the dataframe on only the rows with the Sizes and Countries in the parameter file
7. Look up the related Class Names of the class id's ("H", "M", "L") from the parameter file, into a separate "Class Name" column in the dataframe
8. Remove "Class" column, as we only need "Class ID" column instead (to make it easy to distinguish it from Class Name)
9. Use your "Export_to_csv.ipynb" external program to export the cleaned, filtered dataframe to the output folder ("Output"), the output file's name should be: "Orders_all_periods_cleaned_filtered.csv"
10. Create minimum three Markdown cells which give explanations about the goals of the program's sections
11. At the end, write out the runtime, and a beep sound and a final printed message should notify the user

Upload your final Jupyter notebook (.ipynb) in ZIP!
```

In [1]:
# imports of python modules and notebooks
import os
import pandas as pd

# instead of calling the notebooks inline, most notebooks have exported functions
# import those here
%run Read_csv.ipynb
%run Read_parameters.ipynb
%run Export_to_csv.ipynb
%run Utils.ipynb

## Constants, please adjust them according to your settings

In [2]:
# constants
BASE_PATH = r'c:\Users\nyos\cubix_data_engineer\Python_DA'
HOMEWORK_DIR = 'Homework4'
INPUT_DIR = 'Input'
OUTPUT_DIR = 'Output'
PARAM_FILE = 'Parameters_Homework4.xlsx'
OUT_FILE = 'Orders_all_periods_cleaned_filtered.csv'
param_columndict = {'sizes': 'A:A', 'countries': 'C:C', 'classes': 'E:F'}

In [3]:
FULL_HOMEWORK_PATH = [BASE_PATH, HOMEWORK_DIR]
FULL_INPUT_PATH = FULL_HOMEWORK_PATH + [INPUT_DIR]

## Read parameters

In [4]:
param_pathlist = FULL_HOMEWORK_PATH + [PARAM_FILE]
parameters = read_params(param_pathlist, param_columndict, 'Parameter tables')

## Read and merge every CSV into the main DataFrame

In [5]:
main_df = pd.DataFrame()
for fname in os.listdir(os.path.join(*FULL_INPUT_PATH)):
    main_df = pd.concat((main_df, read_csv(*FULL_INPUT_PATH, fname)), ignore_index=True)
main_df

,OrderDate,Country,LineTotal,ProductName,ProductNumber,Class
0,2011.05.31 0:00,AU,"3399,99","Mountain-100 Silver, 44",BK-M82S-44,H
1,2011.05.31 0:00,CA,"3578,27","Road-150 Red, 62",BK-R93R-62,H
2,2011.05.31 0:00,FR,"3399,99","Mountain-100 Silver, 44",BK-M82S-44,H
3,2011.05.31 0:00,US,"3399,99","Mountain-100 Silver, 44",BK-M82S-44,H
4,2011.05.31 0:00,DE,"2024,994","Mountain-100 Black, 42",BK-M82B-42,H
...,...,...,...,...,...,...
121312,2014.06.30 0:00,US,"21,98",Fender Set - Mountain,FE-6654,L
121313,2014.06.30 0:00,US,"54,99",Hydration Pack - 70 oz.,HY-1023-70,L
121314,2014.06.30 0:00,US,"7,95",Bike Wash - Dissolver,CL-9009,L
121315,2014.06.30 0:00,US,"4,99",Mountain Tire Tube,TT-M928,L


## Clean the DataFrame using a modified variant of Homework02

In [6]:
df = pd.DataFrame(main_df)  # this way we can re-run this cell
%run Clean_order_data.ipynb  # clean df in place
# Clean has many cells that modify df, so it's better to call it like this
# forcing multiple cells into one function would be worse
df

,OrderDate,Country,LineTotal,ProductName,ProductNumber,Class,Size
0,2011-05-31,AU,3399.990,"Mountain-100 Silver, 44",BK-M82S-44,H,44
1,2011-05-31,AU,809.760,"HL Mountain Frame - Black, 48",FR-M94B-48,H,48
2,2011-05-31,AU,4049.988,"Mountain-100 Black, 44",BK-M82B-44,H,44
3,2011-05-31,AU,2039.994,"Mountain-100 Silver, 44",BK-M82S-44,H,44
4,2011-05-31,AU,4079.988,"Mountain-100 Silver, 44",BK-M82S-44,H,44
...,...,...,...,...,...,...,...
121312,2014-06-30,US,21.980,Fender Set - Mountain,FE-6654,L,None
121313,2014-06-30,US,54.990,Hydration Pack - 70 oz.,HY-1023-70,L,None
121314,2014-06-30,US,7.950,Bike Wash - Dissolver,CL-9009,L,None
121315,2014-06-30,US,4.990,Mountain Tire Tube,TT-M928,L,None


## Convert the size into a string
(it was a str by default so does nothing)

In [7]:
parameters['sizes']['Size'] = parameters['sizes']['Size'].astype('str')

## Filter the data for countries and sizes based on the parameter file

In [8]:
country_list = parameters['countries']['Country'].tolist()
size_list = parameters['sizes']['Size'].tolist()

In [9]:
df = df[(df['Country'].isin(country_list)) & (df['Size'].isin(size_list))]

## Do a left join for the classes and clean up the result

In [10]:
merged = pd.merge(df, parameters['classes'], how='left', left_on='Class', right_on='Class ID')
merged.rename(columns={'Class name': 'Class Name'}, inplace=True)
del merged['Class']
merged

,OrderDate,Country,LineTotal,ProductName,ProductNumber,Size,Class ID,Class Name
0,2011-05-31,DE,4293.9240,"Road-150 Red, 56",BK-R93R-56,56,H,High
1,2011-05-31,DE,2146.9620,"Road-150 Red, 56",BK-R93R-56,56,H,High
2,2011-05-31,DE,4293.9240,"Road-150 Red, 56",BK-R93R-56,56,H,High
3,2011-05-31,DE,535.7424,"LL Road Frame - Black, 58",FR-R38B-58,58,L,Low
4,2011-05-31,DE,178.5808,"LL Road Frame - Black, 52",FR-R38B-52,52,L,Low
...,...,...,...,...,...,...,...,...
7387,2014-05-30,GB,1214.8500,"Touring-2000 Blue, 60",BK-T44U-60,60,M,Medium
7388,2014-06-03,GB,53.9900,"Short-Sleeve Classic Jersey, XL",SJ-0194-X,XL,L,Low
7389,2014-06-18,DE,53.9900,"Short-Sleeve Classic Jersey, XL",SJ-0194-X,XL,L,Low
7390,2014-06-22,GB,53.9900,"Short-Sleeve Classic Jersey, XL",SJ-0194-X,XL,L,Low


## Export the result and notify the user in the end

In [11]:
write_csv(merged, *FULL_HOMEWORK_PATH, OUTPUT_DIR, OUT_FILE)

In [12]:
runtime()

'Runtime: 00:00:01.92'

In [13]:
notify_done()

Everything done.
